<a href="https://colab.research.google.com/github/Ctm31/Stock-Twits-Sentiment-Analysis/blob/main/twitsScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # Update package lists
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Package

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import csv

In [ ]:
#base url to scrape
#url = 'https://stocktwits.com/'
url = 'https://stocktwits.com/symbol/PRVB'

# Setting up drive options
options = Options()
options.headless = True
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(options=options)

In [ ]:
from google.colab import userdata
email = userdata.get('stock_twits_email')
password = userdata.get('stock_twits_password')

In [ ]:
# signing in to page
driver.get(url)

time.sleep(1)

driver.find_element(By.CSS_SELECTOR, '.h-\[18px\]').click()

time.sleep(1)

email_input = driver.find_element(By.NAME, 'login')
password_input = driver.find_element(By.NAME, 'password')

email_input.send_keys(email)
password_input.send_keys(password)

time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="Layout"]/div[1]/div[3]/div/div[2]/form/button').click()

time.sleep(3)

#Scrolling down page
for x in range(150):
    time.sleep(1+(x/500))
    ActionChains(driver)\
        .scroll_by_amount(0, 7000-(10*x))\
        .perform()

# Wait until elements load
elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.whitespace-pre-wrap')))

# Get wanted elements
elements = driver.find_elements(By.CSS_SELECTOR, '.whitespace-pre-wrap')
header_elements = driver.find_elements(By.CSS_SELECTOR, '.gap-x-2')
date_elements = driver.find_elements(By.CSS_SELECTOR, '.text-\[11px\]')

In [ ]:
# Getting web element text values, takes a while
elem_text = []

for elem in elements:
    elem_text.append(elem.text)

In [ ]:
# Getting web element date values
dates = []

for date in date_elements:
    dates.append(date.get_attribute('datetime'))

In [ ]:
print(dates[1:10])

['2024-06-25T18:12:19Z', '2024-05-23T13:39:45Z', '2024-05-22T15:54:10Z', '2024-05-20T14:20:18Z', '2024-05-12T13:39:43Z', '2024-05-10T15:14:55Z', '2024-05-06T13:18:07Z', '2024-03-17T03:25:33Z', '2024-04-15T19:31:13Z']


In [ ]:
print(elem_text[7])

$SGMT
 I'm trying to understand why Dennis Hom left last July right after IPO. The dude was the CFO for many many years and led all financing rounds and the Chinese deal.
ir.sagimet.com/news-release...


In [ ]:
# Process elements for data table
elem_text_clean = []

# Separating sentiment from text
for elem in elem_text:
    e_txt = elem.replace('\n', '')
    a_txt = e_txt[:-7]
    end = e_txt[-7:]

    if end == "Bullish":
        elem_text_clean.append([a_txt, "Bullish"])
    elif end == "Bearish":
        elem_text_clean.append([a_txt, "Bearish"])
    else:
        elem_text_clean.append([e_txt])

# Splitting date and time
dates_clean = []
for date in dates:
    if (date is not None):
        dates_clean.append(date.split('T'))
    else:
        dates_clean.append(["D", "T"])

# Splitting name from datetime
headers = [head.text.split('\n') for head in header_elements]

print(f"scraped: {len(elem_text)} results!")
#print(headers[1:10])
#print(elem_text[1:5])

driver.quit()

scraped: 2569 results!


In [ ]:
print(dates_clean[1:10])

[['2024-06-25', '18:12:19Z'], ['2024-05-23', '13:39:45Z'], ['2024-05-22', '15:54:10Z'], ['2024-05-20', '14:20:18Z'], ['2024-05-12', '13:39:43Z'], ['2024-05-10', '15:14:55Z'], ['2024-05-06', '13:18:07Z'], ['2024-03-17', '03:25:33Z'], ['2024-04-15', '19:31:13Z']]


In [ ]:
print(elem_text_clean[7])

["$SGMT I'm trying to understand why Dennis Hom left last July right after IPO. The dude was the CFO for many many years and led all financing rounds and the Chinese deal.ir.sagimet.com/news-release..."]


In [ ]:
# writing scraped data to csv

filename1 = 'stockTwits_names_PRVB.csv'
filename2 = 'stockTwits_text_PRVB.csv'
filename3 = 'stockTwits_dates_PRVB.csv'


# writing to csv file
with open(filename1, 'w+') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fields
    csvwriter.writerow(['Name', 'Date/Time'])

    # writing the data rows
    csvwriter.writerows(headers)

with open(filename2, 'w+', encoding='utf-8') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fields
    csvwriter.writerow(['Text', 'Sentiment'])

    # writing the data rows
    csvwriter.writerows(elem_text_clean)

with open(filename3, 'w+', encoding='utf-8') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fields
    csvwriter.writerow(['Date', 'Time'])

    # writing the data rows
    csvwriter.writerows(dates_clean)
